In [19]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


In [20]:
df = pd.read_csv("./data/importacion.csv", parse_dates=["Fecha"])
df.set_index("Fecha", inplace=True)


In [21]:
# Preparacion gasolina regular
regular_series = df["Gasolina regular"]

regular_split = int(len(regular_series) * 0.7)
regular_train = regular_series.iloc[:regular_split] 
regular_test = regular_series.iloc[regular_split:]

regular_scaler = MinMaxScaler()
regular_train_scaled = regular_scaler.fit_transform(regular_train.values.reshape(-1, 1))
regular_test_scaled = regular_scaler.transform(regular_test.values.reshape(-1, 1))

In [22]:
# Preparacion gasolina superior
super_series = df["Gasolina superior"]

super_split = int(len(regular_series) * 0.7)
super_train = super_series.iloc[:super_split] 
super_test = super_series.iloc[super_split:]

super_scaler = MinMaxScaler()
super_train_scaled = super_scaler.fit_transform(super_train.values.reshape(-1, 1))
super_test_scaled = super_scaler.transform(super_test.values.reshape(-1, 1))

In [23]:
def create_sequences(data, window=12):
    X, y = [], []
    for i in range(window, len(data)):
        X.append(data[i-window:i])
        y.append(data[i])
    return np.array(X), np.array(y)


In [24]:
# Para Gasolina regular
window_size = 12
X_reg_train, y_reg_train = create_sequences(regular_train_scaled, window=window_size)
X_reg_test,  y_reg_test  = create_sequences(regular_test_scaled,  window=window_size)

# Para Gasolina superior
X_sup_train, y_sup_train = create_sequences(super_train_scaled, window=window_size)
X_sup_test,  y_sup_test  = create_sequences(super_test_scaled,  window=window_size)

In [25]:
# Redimensiona al formato [muestras, timesteps, features]
X_reg_train = X_reg_train.reshape(-1, window_size, 1)
X_reg_test  = X_reg_test.reshape(-1,  window_size, 1)
X_sup_train = X_sup_train.reshape(-1, window_size, 1)
X_sup_test  = X_sup_test.reshape(-1,  window_size, 1)

In [26]:
# Convierte NumPy → Torch Tensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_reg_train_t = torch.tensor(X_reg_train, dtype=torch.float32).to(device)
y_reg_train_t = torch.tensor(y_reg_train, dtype=torch.float32).to(device)
X_reg_test_t  = torch.tensor(X_reg_test,  dtype=torch.float32).to(device)
y_reg_test_t  = torch.tensor(y_reg_test,  dtype=torch.float32).to(device)


In [27]:
# Dataset y DataLoader
batch_size = 32

train_ds = TensorDataset(X_reg_train_t, y_reg_train_t)
test_ds  = TensorDataset(X_reg_test_t,  y_reg_test_t)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size)

In [28]:

# Define dos arquitecturas de LSTM
class LSTMModelA(nn.Module):
    def __init__(self, input_size=1, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        last = out[:, -1, :]
        y = self.dropout(last)
        y = self.fc(y)
        return y

In [29]:
class LSTMModelB(nn.Module):
    def __init__(self, input_size=1, hidden_sizes=(100,50)):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size,  hidden_sizes[0], batch_first=True)
        self.dropout1 = nn.Dropout(0.3)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.fc     = nn.Linear(hidden_sizes[1], 1)
        
    def forward(self, x):
        out, _ = self.lstm1(x)
        out     = self.dropout1(out)
        out, _ = self.lstm2(out)
        last    = out[:, -1, :]
        return self.fc(last)

In [30]:
# Bucle de entrenamiento y validación
def train_model(model, train_loader, test_loader, epochs=50, lr=1e-3):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_loss = float('inf')
    patience, counter = 5, 0
    
    for epoch in range(1, epochs+1):
        # -- Entrenamiento --
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds.squeeze(), yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)
        train_loss = running_loss / len(train_loader.dataset)
        
        # -- Validación --
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for xb, yb in test_loader:
                preds = model(xb)
                loss = criterion(preds.squeeze(), yb)
                running_loss += loss.item() * xb.size(0)
        val_loss = running_loss / len(test_loader.dataset)
        
        print(f"Epoch {epoch:02d} — Train MSE: {train_loss:.4f} — Val MSE: {val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), "best_model.pth")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping.")
                break
    
    model.load_state_dict(torch.load("best_model.pth"))
    return model

In [31]:
# Ejecución ejemplar
model_a = LSTMModelA()
print("Entrenando Modelo A")
model_a = train_model(model_a, train_loader, test_loader)

model_b = LSTMModelB()
print("Entrenando Modelo B")
model_b = train_model(model_b, train_loader, test_loader)


Entrenando Modelo A
Epoch 01 — Train MSE: 0.1747 — Val MSE: 1.1837
Epoch 02 — Train MSE: 0.1123 — Val MSE: 0.9472
Epoch 03 — Train MSE: 0.0678 — Val MSE: 0.6721
Epoch 04 — Train MSE: 0.0518 — Val MSE: 0.4472
Epoch 05 — Train MSE: 0.0631 — Val MSE: 0.4865


c:\Users\MSI\Desktop\info\Escritorio\UVG\8vo Semestre\Data Science\DSvenv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\MSI\Desktop\info\Escritorio\UVG\8vo Semestre\Data Science\DSvenv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\MSI\Desktop\info\Escritorio\UVG\8vo Semestre\Data Science\DSvenv\Lib\site-packages\torch\nn\modules\loss.py:610: UserWarning: Using a target size (torch.Size([12, 1])) that is different to the input s

Epoch 06 — Train MSE: 0.0529 — Val MSE: 0.5762
Epoch 07 — Train MSE: 0.0505 — Val MSE: 0.6581
Epoch 08 — Train MSE: 0.0517 — Val MSE: 0.6683
Epoch 09 — Train MSE: 0.0519 — Val MSE: 0.6663
Early stopping.
Entrenando Modelo B
Epoch 01 — Train MSE: 0.0733 — Val MSE: 0.6763
Epoch 02 — Train MSE: 0.0484 — Val MSE: 0.5099
Epoch 03 — Train MSE: 0.0521 — Val MSE: 0.6139
Epoch 04 — Train MSE: 0.0515 — Val MSE: 0.7304
Epoch 05 — Train MSE: 0.0509 — Val MSE: 0.6632
Epoch 06 — Train MSE: 0.0488 — Val MSE: 0.6333
Epoch 07 — Train MSE: 0.0485 — Val MSE: 0.6116
Early stopping.
